In [ ]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import pywt
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt 
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy.io import loadmat
from entropy import *
from scipy import signal


# %%
data = loadmat('BCI_2003_datasetIII.mat')
X = data['x_train']
y = data['y_train']


# %%
"""
Feature Selection and Pre-processing

"""
# Remove 0-3s from the signal


# %%
X = X[384:, :, :]
# sliding window PSE
def window_pse(signal, size= 128):
    zero = np.zeros(len(signal))
    for i in range(len(signal)):
        zero[i] = spectral_entropy(np.abs(signal[i: size+i]), 128, method='welch', normalize=True)
    return zero

def moving_average(x, w=16):
    return np.convolve(x, np.ones(w), 'valid') / w


# %%
i = moving_average(X[:,0,49])
n =  i.shape[0]
print(n)


# %%
n =  i.shape[0]
X_tf = np.zeros((n,3,140))
for i in range(140):
    for j in range(3):
        X_tf[:,j,i] = moving_average(X[:,j,i]) 


# %%
for i in range(140):
    for j in range(3):
        X_tf[:,j,i] = window_pse(X_tf[:,j,i]) 


# %%
X_tf = X_tf[:700,:,:]


# %%



# %%
left_index = np.argwhere(y == 1)
right_index = np.argwhere(y == 2)

X_left = X[:,:,left_index[:,0]]
# 
c3l_mean = np.mean(X_left[:,0,:], axis=1)
czl_mean = np.mean(X_left[:,1,:], axis=1)
c4l_mean = np.mean(X_left[:,2,:], axis=1)

X_right = X[:,:,right_index[:,0]]
#
c3r_mean = np.mean(X_right[:,0,:], axis=1)
czr_mean = np.mean(X_right[:,1,:], axis=1)
c4r_mean = np.mean(X_right[:,2,:], axis=1)


# %%
c3l_mean = moving_average(c3l_mean)
c3r_mean = moving_average(c3r_mean)

pse3_left = window_pse(c3l_mean)
pse3_right = window_pse(c3r_mean)

t1, f1 , sft3_left = signal.stft(c3l_mean, 265, nperseg=100)
t2, f1, sft3_right= signal.stft(c3r_mean, 256, nperseg=100)

f, ax = plt.subplots(2, figsize=(15,15) ,sharey=False)

ax[0].plot(t1, np.abs(sft3_left), color='black')
ax[0].plot(t2, np.abs(sft3_right), color='red')
ax[0].set_title('Short Time FFT')

ax[0].set(xlabel='Frequency', ylabel='Magnitude')
ax[0].legend(('left', 'right'))

ax[1].plot(pse3_left, color='black')
ax[1].plot(pse3_right, color='red')
ax[1].set_title('Power Spectral Entropy')

ax[1].set(xlabel='Time', ylabel='Power Spectral Entropy')
ax[1].legend(('left', 'right'))

xlables = ['Frequency', 'Time']
ylables = ['Magnitude', 'Entropy']

plt.show()


# %%
"""
Feature Extraction

"""
X_tf = np.zeros((140,3))
for i in range(140):
    for j in range(3):
        x_mean = moving_average(X[:,j,i])
        pse = window_pse(x_mean)
        xx = pse[0:400]
        ft = xx[~np.isnan(xx)]
        X_tf[i,j] = np.mean(ft)
#
X_f = np.zeros((140,3))

for i in range(140):
    for j in range(3):
        yl = X_tf[:,j,i][~np.isnan(X_tf[:,j,i])]
        X_f[i,j] = np.mean(yl)
       


# %%

# PCA Decomposion
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X_tf)

X_pca = pca.transform(X_tf)


# %%
"""
Train/Test Split

"""
X_train, X_test, y_train, y_test = train_test_split(X_pca , y, test_size=0.33, random_state=42)


# %%
"""
Non-linear Support Vector Machine IMPLEMENTATION

"""


# %%
classifier = SVC(kernel='rbf', C=1).fit(X_train, y_train)
classifier.fit(X_train, y_train)


# %%

from sklearn.metrics import plot_confusion_matrix


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(classifier, X_test, y_test,
                                 display_labels=['left', 'right'],
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()


# %%
from sklearn import metrics
y_pred = classifier.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


# %%
"""
KNN
"""
from sklearn.neighbors import KNeighborsClassifier

k_range = range(1,26)
scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y2_pred = knn.predict(X_test)
    scores.append(metrics.accuracy_score(y_test, y2_pred))


# %%
mx = max(scores)
n_max  = scores.index(mx) + 1
print(mx, n_max)


# %%
knn = KNeighborsClassifier(n_neighbors=n_max)
knn.fit(X_train, y_train)


# %%
titles_options = [("Confusion matrix", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(knn, X_test, y_test,
                                 display_labels=['left', 'right'],
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()


# %%
y_pred_knn = knn.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_knn))


# %%
"""
